In [ ]:
!nvidia-smi

Fri Dec 29 07:04:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.36.0
Uninstalling transformers-4.36.0:
  Successfully uninstalled transformers-4.36.0
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.9 MB/s eta 0:00:00
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/f7/fc/c55e5a2da345c9a24aa2e1e0f60eb2ca290b6a41be82da03a6d4baec4f99/accelerate-0.25.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 62.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.5 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
model_ckpt = "DaviadiAF/T5-Small_AbsSumm_XSumCNN"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
dataset_pubmed = load_dataset("EdinburghNLP/xsum")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_pubmed

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [ ]:
dataset_pubmed['train'] = dataset_pubmed['train'][:5000]

In [ ]:
dataset_pubmed["train"]["document"][1]

'A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.\nAs they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.\nOne of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.\nThe driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.\nBoth groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.\nPolice have appealed for information about the attack.\nInsp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.\n"While the exact cause is still under investigation, it is thought that the fire was started deliberately."'

In [ ]:
dataset_pubmed["train"]["summary"][1]

'Two tourist buses have been destroyed by fire in a suspected arson attack in Belfast city centre.'

In [ ]:
split_lengths = [len(dataset_pubmed[split])for split in dataset_pubmed]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_pubmed['train'].keys()}")
print("\nArticle:")

print(dataset_pubmed["test"][1]["document"])

print("\nAbstract:")

print(dataset_pubmed["test"][1]["summary"])

Split lengths: [3, 11332, 11334]
Features: dict_keys(['document', 'summary', 'id'])

Article:
Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday.
Detectives said three firearms, ammunition and a five-figure sum of money were recovered.
A 26-year-old man who was arrested and charged appeared at Edinburgh Sheriff Court on Thursday.

Abstract:
A man has appeared in court after firearms, ammunition and cash were seized by police in Edinburgh.


In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['document'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [ ]:
from datasets import Dataset

In [ ]:
for split in dataset_pubmed:
     dataset_pubmed_pt = dataset_pubmed[split]  # Access the split

     # Only convert to Dataset if it's not already one
     if not isinstance(dataset_pubmed_pt, Dataset):
        dataset_pubmed_pt = Dataset.from_dict(dataset_pubmed_pt)

     dataset_pubmed_pt = dataset_pubmed_pt.map(convert_examples_to_features, batched=True)  # Apply map


  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
dataset_pubmed_pt

Dataset({
    features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 11334
})

In [ ]:
dataset_pubmed_pt["attention_mask"][1]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='t5-pubmed', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
dataset_pubmed_pt.column_names

['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels']

In [ ]:
from datasets import DatasetDict

In [ ]:
from datasets import Dataset

# Assuming the entire dataset is currently in `dataset_pubmed_pt`
train_split = dataset_pubmed_pt.train_test_split(test_size=0.2)  # Split into 80% train, 20% test
dataset_pubmed_pt = DatasetDict({
    'train': train_split['train'],
    'test': train_split['test'],
})


In [ ]:
# Assuming the "train" and "test" splits are now available
validation_split = dataset_pubmed_pt['train'].train_test_split(test_size=0.2)  # Split "train" into 80% train, 20% validation
dataset_pubmed_pt = DatasetDict({
    'train': validation_split['train'],
    'test': dataset_pubmed_pt['test'],
    'validation': validation_split['test']  # Assign the newly created validation split
})

In [ ]:
trainer = Trainer(model=model_t5, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pubmed_pt["test"],
                  eval_dataset=dataset_pubmed_pt["validation"])

In [ ]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=141, training_loss=2.890908288617506, metrics={'train_runtime': 167.0561, 'train_samples_per_second': 13.57, 'train_steps_per_second': 0.844, 'total_flos': 289783979016192.0, 'train_loss': 2.890908288617506, 'epoch': 1.0})

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_pubmed['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'document', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f't5'] )

100%|██████████| 5/5 [00:03<00:00,  1.43it/s]


,rouge1,rouge2,rougeL,rougeLsum
t5,0.024194,0.0,0.024059,0.02387


In [ ]:
## Save model
model_t5.save_pretrained("t5-pubmed-model")

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/tokenizer.json')

In [ ]:
#Load

tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_pubmed["test"][0]["document"]
print("Document:")
print(sample_text)


Document:
Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.
Andrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for 

In [ ]:
reference = dataset_pubmed["test"][0]["summary"]

print("\nReference Summary:")
print(reference)




Reference Summary:
There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.


In [ ]:
print(gen_kwargs)

{'length_penalty': 0.8, 'num_beams': 8, 'max_length': 128}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your fine-tuned model
model_pegasus_finetuned = AutoModelForSeq2SeqLM.from_pretrained("t5-pubmed-model")
tokenizer_finetuned = AutoTokenizer.from_pretrained("tokenizer")

# Tokenize the input sequence
tokenized_input = tokenizer_finetuned(sample_text, max_length=1024, truncation=True, return_tensors="pt")

# Get the input IDs and attention mask
input_ids = tokenized_input["input_ids"]
attention_mask = tokenized_input["attention_mask"]

# Generate summary for each chunk
max_chunk_size = 512  # Choose a suitable chunk size
chunk_start = 0
generated_summaries = []

while chunk_start < len(input_ids[0]):
    # Get the current chunk
    input_chunk = input_ids[:, chunk_start : chunk_start + max_chunk_size]
    attention_mask_chunk = attention_mask[:, chunk_start : chunk_start + max_chunk_size]

    # Generate summary for the current chunk
    summary_chunk = model_pegasus_finetuned.generate(
        input_ids=input_chunk,
        attention_mask=attention_mask_chunk,
        max_length=128,
        length_penalty=0.8,
        num_beams=4,
    )

    # Decode and append the summary
    decoded_summary_chunk = tokenizer_finetuned.decode(summary_chunk[0], skip_special_tokens=True)
    generated_summaries.append(decoded_summary_chunk)

    # Move to the next chunk
    chunk_start += max_chunk_size

# Combine the generated summaries into a single summary
final_summary = " ".join(generated_summaries)
print("Generated Summary:", final_summary)


Generated Summary: The need for accommodation for prison leavers is "chronic", a Welsh prison worker has said. A homeless charity has found a stable accommodation with a homeless charity, Emmaus South Wales.
